In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 定义简单的两层神经网络
class RZDecoderNet(nn.Module):
    def __init__(self, input_size):
        super(RZDecoderNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 16)
        self.fc2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

# 从文件中加载数据
def load_data_from_file(file_path, bit_length):
    inputs = []
    outputs = []
    with open(file_path, 'r') as f:
        for line in f:
            values = [float(x) for x in line.strip().split()]
            inputs.append(values[:bit_length])  # 前 bit_length 个值为输入信号
            outputs.append(values[bit_length])  # 最后一个值为输出标签

    inputs = torch.tensor(inputs, dtype=torch.float32)
    outputs = torch.tensor(outputs, dtype=torch.float32).unsqueeze(1)  # 添加维度以匹配网络输出
    return inputs, outputs

# 训练 RZ 解码网络
def train_rz_decoder(model, train_loader, num_epochs=20, learning_rate=0.01):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# 使用加载的模型进行预测
def predict_with_model(model, inputs):
    with torch.no_grad():  # 禁用梯度计算
        outputs = model(inputs)
        predictions = (outputs >= 0.5).float()  # 阈值设为 0.5，输出 0 或 1
    return predictions

# 参数
bit_length = 1  # 每个比特的长度为 1

# 文件路径
train_file_path = r'E:\gitprogram\data\testdata\train.txt'
test_file_path = r'E:\gitprogram\data\testdata\test.txt'
exp_file_path = r'E:\gitprogram\data\testdata\exp.txt'

# 从文件加载训练和测试数据
train_inputs, train_labels = load_data_from_file(train_file_path, bit_length)
test_inputs, test_labels = load_data_from_file(test_file_path, bit_length)

# 创建数据加载器
train_dataset = TensorDataset(train_inputs, train_labels)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

# 初始化网络
input_size = bit_length
model = RZDecoderNet(input_size)

# 训练网络
train_rz_decoder(model, train_loader, num_epochs=20, learning_rate=0.01)

# 保存训练后的模型
torch.save(model.state_dict(), 'trained_model.pth')

# 加载已训练的模型
model.load_state_dict(torch.load('trained_model.pth'))
model.eval()  # 设置为评估模式

# 读取实验数据
exp_inputs, _ = load_data_from_file(exp_file_path, bit_length)

# 进行预测
predicted_outputs = predict_with_model(model, exp_inputs)

# 输出预测结果
print("Predicted outputs:", predicted_outputs.squeeze().int())


Epoch [1/20], Loss: 0.6881
Epoch [2/20], Loss: 0.6790
Epoch [3/20], Loss: 0.6704
Epoch [4/20], Loss: 0.6621
Epoch [5/20], Loss: 0.6541
Epoch [6/20], Loss: 0.6462
Epoch [7/20], Loss: 0.6383
Epoch [8/20], Loss: 0.6305
Epoch [9/20], Loss: 0.6225
Epoch [10/20], Loss: 0.6146
Epoch [11/20], Loss: 0.6065
Epoch [12/20], Loss: 0.5986
Epoch [13/20], Loss: 0.5905
Epoch [14/20], Loss: 0.5823
Epoch [15/20], Loss: 0.5741
Epoch [16/20], Loss: 0.5659
Epoch [17/20], Loss: 0.5577
Epoch [18/20], Loss: 0.5494
Epoch [19/20], Loss: 0.5411
Epoch [20/20], Loss: 0.5329
Predicted outputs: tensor([1, 1, 1], dtype=torch.int32)
